In [34]:
%notebook code_logs/Encoder_logs.ipynb

In [2]:
!pip install torch
!pip install tensorflow
!pip install jax
!pip install transformers
!pip install nbformat

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/ab/6a/0debe1ec3c63b1fd7487ec7dd8fb1adf19898bef5a8dc151265d79ffd915/torch-2.1.0-cp310-none-macosx_11_0_arm64.whl.metadata
  Using cached torch-2.1.0-cp310-none-macosx_11_0_arm64.whl.metadata (24 kB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da2

In [39]:
import pandas as pd
import numpy as np

In [6]:
import deepchem as dc
import torch
from rdkit import Chem

In [ ]:
!pip install --pre deepchem[torch]

In [26]:
drug_smiles=pd.read_csv("Data/SMILEinchi.csv",usecols=['drug_id','smiles'])

In [36]:
smiles_list=drug_smiles['smiles'].values.tolist()
print(len(smiles_list))

395


In [37]:
#from deepchem.feat import HuggingFaceFeaturizer
from transformers import RobertaTokenizerFast
hf_tokenizer = RobertaTokenizerFast.from_pretrained("seyonec/PubChem10M_SMILES_BPE_60k")
featurizer = dc.feat.HuggingFaceFeaturizer(tokenizer=hf_tokenizer)
result = featurizer.featurize(smiles_list)

In [45]:
smiles = drug_smiles['smiles'].values.tolist()
X = np.array(smiles)
dataset = dc.data.NumpyDataset(X)
trans = dc.trans.FeaturizationTransformer(dataset, dc.feat.HuggingFaceFeaturizer(hf_tokenizer))
dataset = trans.transform(dataset)

In [46]:
print(dataset.X)

[{'input_ids': [0, 39, 2], 'attention_mask': [1, 1, 1]}
 {'input_ids': [0, 262, 2], 'attention_mask': [1, 1, 1]}]
